<a href="https://colab.research.google.com/github/tobiasuruali/WebScraper_PolAds/blob/main/code/05_gpt_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount your Google Drive Folders
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip -qqq install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00
time: 454 µs (started: 2023-04-28 14:02:20 +00:00)


In [3]:
import os
os.chdir('/content/drive/MyDrive/Google_Pol_Ads')
print(os.listdir())
print(os.getcwd())

['data', 'env']
/content/drive/MyDrive/Google_Pol_Ads
time: 5.84 ms (started: 2023-04-28 14:02:20 +00:00)


In [7]:
import pandas as pd

# Load the CSV file from Google Drive
df_trump = pd.read_csv('data/data_processed/df_trump_text.csv')

# Load the CSV file from Google Drive
df_biden = pd.read_csv('data/data_processed/df_biden_text.csv')

# Display the first 5 rows of the DataFrame
df_trump.head()


,ad_id,date_range_start,ad_url,advertiser_id,advertiser_name,youtube_url,ad_text
0,CR06816033659036893185,2020-03-12,https://adstransparency.google.com/advertiser/...,AR12365610929977556993,BIDEN FOR PRESIDENT,https://www.youtube.com/watch?v=1tPFGsOivHM&em...,"Cuando Joe Biden se enfrentó al NRA, logró qu..."
1,CR17518932181727051777,2020-10-14,https://adstransparency.google.com/advertiser/...,AR12365610929977556993,BIDEN FOR PRESIDENT,https://www.youtube.com/watch?v=_C-WvLV5UII&em...,People have been denied the right to vote. Fi...
2,CR17406877790644469761,2020-09-08,https://adstransparency.google.com/advertiser/...,AR12365610929977556993,BIDEN FOR PRESIDENT,https://www.youtube.com/watch?v=MSZO8zPUYL0&em...,My future and the future of everyone my age d...
3,CR17616576648293908481,2020-07-16,https://adstransparency.google.com/advertiser/...,AR12365610929977556993,BIDEN FOR PRESIDENT,https://www.youtube.com/watch?v=uLwYRdUxNcY&em...,"After a car crash changed his life forever, J..."
4,CR14354267786883104769,2020-08-11,https://adstransparency.google.com/advertiser/...,AR12365610929977556993,BIDEN FOR PRESIDENT,https://www.youtube.com/watch?v=7ShIz1wlcSw&em...,Leadership is built on trust. Joe Biden led t...


time: 97.7 ms (started: 2023-04-28 14:03:52 +00:00)


In [ ]:
df_biden.head()

# Sentiment Analysis

As Sentiment Analysis we will use a novice approach. 
We will use OpenAi's GPT-3 Model to do the Sentiment Analysis for us. 

Giving it a prompt as displayed in [Andrew NG's New Class on Prompt Engineering](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/5/inferring) we will make the model output us a Sentiment Score between -1 and 1. 

In [8]:
!pip -qqq install openai
!pip -qqq install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.1 MB/s eta 0:00:00
time: 16.3 s (started: 2023-04-28 14:05:20 +00:00)


In [16]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("env/.env")) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

time: 428 ms (started: 2023-04-28 14:15:46 +00:00)


In [18]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

time: 1.35 ms (started: 2023-04-28 14:16:13 +00:00)


Now we'll create a prompt to let GPT3.5 create our sentiment analysis

In [27]:
trump_text_test = print(df_trump.loc[2,'ad_text'])

 Biden's plan is not a solution. It is a virtual surrender. Biden's plan for the China virus is to shut down the entire U.S. economy. I would shut it down. No miracle is coming. If I were president today, I would roll back the reopenings in a number of places. Biden wants to surrender our country to the virus. He wants to surrender our families to the pilot left-wing mob. And he wants to surrender our jobs to China. I would shut it down. I'm Donald Trump and I approve this message. I'm Donald Trump and I approve this message.
time: 1.64 ms (started: 2023-04-28 14:19:03 +00:00)


In [32]:
prompt = f"""
You are a Sentiment Analysis Expert in Political Advertising.
What is the sentiment of the following Political Campaing Text?
​
On a scale of -1 to 1, -1 being negative and 1 positive. The number can include all 0.1 decimal numbers. Reply without explanation, ONLY a single number!
Your output shall never be 0.

Advertisment Text: '''{trump_text_test}'''
"""

response = get_completion(prompt)
print(response)

0.1
time: 1.03 s (started: 2023-04-28 14:21:54 +00:00)


In [33]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

trump_text_test = print(df_trump.loc[2,'ad_text'])

prompt = f"""
You are a Sentiment Analysis Expert in Political Advertising.
What is the sentiment of the following Political Campaing Text?
​
On a scale of -1 to 1, -1 being negative and 1 positive. The number can include all 0.1 decimal numbers. Reply without explanation, ONLY a single number!
Your output shall never be 0.

Advertisment Text: '''{trump_text_test}'''
"""

response = get_completion(prompt)
print(response)

time: 752 µs (started: 2023-04-28 14:22:39 +00:00)


time: 486 ms (started: 2023-04-28 14:22:39 +00:00)
